### Document of PrimeHub python SDK

Visit here https://github.com/InfuseAI/primehub-python-sdk and you can know how to use the PrimeHub SDK to do the setting.

### Prerequisite

#### 1. Parameter Setting

In [1]:
# Airgapped mode
AIRGAPPED = True

In [2]:
# PrimeHub Cluster
PRIMEHUB_CLUSTER = '<primehub_cluster_url>'

In [3]:
# Current group
current_group = "End-to-end-tutorial"

In [4]:
# End to end tutorial folder name (PS: Please put it into the group volume.)
e2e_folder_name = "primehub-screw-detection"

In [5]:
# Parameter for learning rate.
base_learning_rate = 0.1

#### 2. Install package: primehub-python-sdk

In [6]:
if not AIRGAPPED:
    !pip install primehub-python-sdk==0.3.8

### I. Getting Started with PrimeHub Python SDK

#### 1. Generate config file.

In [7]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if not os.path.isfile(os.path.join(os.getenv("HOME"), ".primehub/config.json")):
    ph.config.generate(PRIMEHUB_CLUSTER)

#### 2. Setting PrimeHub config

In [8]:
ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
    ph.config.set_group(current_group)
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("PrimeHub Python SDK couldn't get the group information, please check the configuration.")

PrimeHub Python SDK setup successfully
Current Group: {'id': 'dc6a0f50-2679-4d6b-b819-8da1b2e1b0f9', 'name': 'End-to-end-tutorial', 'displayName': 'End to end tutorial'}


In [9]:
ph.me.me()

{'id': '35985b6b-21c9-4362-902d-4d6bb89074bf',
 'username': 'simonliu@infuseai.io',
 'firstName': 'Simon',
 'lastName': 'Liu',
 'email': 'simonliu@infuseai.io',
 'isAdmin': True}

In [10]:
for cmd in ph.get_all_commands():
    print(cmd)

admin
apps
apptemplates
config
datasets
deployments
files
groups
images
instancetypes
jobs
me
models
notebooks
recurring-jobs
secrets
version
volumes


### II. Training Model.

#### 1. List the docker images environments.

In [11]:
import pandas as pd
df_list = pd.DataFrame.from_records(list(ph.images.list()))
df_list

,id,name,displayName,description,useImagePullSecret,spec
0,tf-2.5,tf-2.5,TensorFlow 2.5 with PrimeHub extension,TensorFlow 2.5 with PrimeHub extension,None,{'description': 'TensorFlow 2.5 with PrimeHub ...
1,tf-1.15,tf-1.15,TensorFlow 1.15 with PrimeHub extension,TensorFlow 1.15 with PrimeHub extension,None,{'description': 'TensorFlow 1.15 with PrimeHub...
2,pytorch-1.10,pytorch-1.10,PyTorch 1.10 with PrimeHub extension,PyTorch 1.10 with PrimeHub extension,None,{'description': 'PyTorch 1.10 with PrimeHub ex...
3,tf-2.8,tf-2.8,TensorFlow 2.8 with PrimeHub extension,TensorFlow 2.8 with PrimeHub extension,None,{'description': 'TensorFlow 2.8 with PrimeHub ...
4,base-notebook,base-notebook,base-notebook,base-notebook,None,"{'description': 'base-notebook', 'displayName'..."
5,global,global,global,,,"{'description': '', 'displayName': 'global', '..."


#### 2. Environment variable setting

In [12]:
current_group = current_group.lower()

In [13]:
from datetime import datetime
now_time = datetime.today().strftime('%Y%m%d_%H%M%S')
now_time

'20230104_085938'

#### 3. Submit Notebook as a job.

In [14]:
config = {
    "instanceType": "cpu-2",
    "image": "tf-2.5",
    "displayName": "primehub screw detection",
    "command": "cd /home/jovyan/{}/{}/ && \
                pip install -r jupyter-image/requirements.txt &&\
                papermill /home/jovyan/{}/{}/tutorial_screw_training_airgapped.ipynb \
                /home/jovyan/{}/{}/tutorial_screw_training_airgapped-{}.ipynb \
                -p base_learning_rate {}".format(
                    current_group, e2e_folder_name, current_group, e2e_folder_name, current_group, \
                    e2e_folder_name, now_time, base_learning_rate)
}

training_job = ph.jobs.submit(config)
print(training_job)

{'id': 'job-202301040059-vua006', 'displayName': 'primehub screw detection', 'cancel': False, 'command': 'cd /home/jovyan/end-to-end-tutorial/primehub-screw-detection/ &&                 pip install -r jupyter-image/requirements.txt &&                papermill /home/jovyan/end-to-end-tutorial/primehub-screw-detection/tutorial_screw_training_airgapped.ipynb                 /home/jovyan/end-to-end-tutorial/primehub-screw-detection/tutorial_screw_training_airgapped-20230104_085938.ipynb                 -p base_learning_rate 0.1', 'groupId': 'dc6a0f50-2679-4d6b-b819-8da1b2e1b0f9', 'groupName': 'End-to-end-tutorial', 'image': 'tf-2.5', 'instanceType': {'id': 'cpu-2', 'name': 'cpu-2', 'displayName': 'CPU 2', 'cpuLimit': 2, 'memoryLimit': 10, 'gpuLimit': 0}, 'userId': '35985b6b-21c9-4362-902d-4d6bb89074bf', 'userName': 'simonliu@infuseai.io', 'phase': 'Pending', 'reason': None, 'message': None, 'createTime': '2023-01-04T00:59:38Z', 'startTime': None, 'finishTime': None, 'recurrence': None}


### III. Model Registry

#### 1. Variable setting

In [15]:
# Mlflow run id
run_id = "<mlflow-run-id>"

In [16]:
# The name of model registry
model_registry_name = "screw-model"

#### 2. Register the model

In [17]:
import mlflow

model_uri = "runs:/{}/model".format(run_id)
mv = mlflow.register_model(model_uri, model_registry_name)

Registered model 'screw-model' already exists. Creating a new version of this model...
2023/01/04 08:59:39 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: screw-model, version 9
Created version '9' of model 'screw-model'.


#### 3. Get the model uri link

In [18]:
model_uri_link = "models:/{}/{}".format(mv.name, mv.version)
model_uri_link

'models:/screw-model/9'

### IV. Model Management

#### 1. List all register models

In [19]:
models = ph.models.list()
for model in models:
    if model["name"] == model_registry_name:
        print(model)

{'name': 'screw-model', 'creationTimestamp': datetime.datetime(2022, 9, 19, 15, 32, 2), 'lastUpdatedTimestamp': datetime.datetime(2023, 1, 4, 8, 59, 39), 'description': None, 'latestVersion': '9'}


#### 2. Get detail information by model name


In [ ]:
ph.models.get(model_registry_name)

### V. Model Deployment

#### 1. Choose the model version for deploying

In [21]:
model_version = 3

#### 2. List all deployments

In [ ]:
ph.deployments.list()

#### 3. Variable Setting

In [23]:
deploy_name = "<name>"

#### 4. Create a deployment

In [24]:
config = {
    "id": "deployment-screw-detection-{}".format(deploy_name),
    "name": "deployment-screw-detection-{}".format(deploy_name),
    "modelImage": "infuseaidev/tensorflow2-prepackaged:screw-classification",
    "modelURI": model_uri_link,
    "instanceType": "cpu-1",
    "replicas": 1
}

deployment = ph.deployments.create(config)
print(deployment)

{'id': 'deployment-screw-detection-simon', 'name': 'deployment-screw-detection-simon', 'modelImage': 'infuseaidev/tensorflow2-prepackaged:screw-classification', 'imagePullSecret': None, 'description': None, 'replicas': 1, 'stop': False, 'endpointAccessType': 'public', 'endpointClients': [], 'status': 'Deploying', 'endpoint': None, 'availableReplicas': None, 'message': None, 'pods': []}


#### 5. See the deployment information(Not ready)

In [25]:
deployment_info = ph.deployments.get(deployment['id'])
deployment_info

{'id': 'deployment-screw-detection-simon',
 'status': 'Deploying',
 'message': None,
 'name': 'deployment-screw-detection-simon',
 'description': None,
 'updateMessage': None,
 'metadata': None,
 'stop': False,
 'userName': 'simonliu@infuseai.io',
 'groupName': 'End-to-end-tutorial',
 'endpoint': None,
 'modelImage': 'infuseaidev/tensorflow2-prepackaged:screw-classification',
 'modelURI': 'models:/screw-model/9',
 'replicas': 1,
 'availableReplicas': None,
 'imagePullSecret': None,
 'instanceType': {'name': 'cpu-1'},
 'creationTime': '2023-01-04T00:59:40Z',
 'lastUpdatedTime': '2023-01-04T00:59:40.317Z',
 'pods': [],
 'env': None,
 'endpointAccessType': 'public',
 'endpointClients': []}

#### 6. Wait for ready

In [ ]:
# Wait for finishing deploy
ph.deployments.wait(deployment_info['id'])

#### 7. See the deployment information(Ready)

In [ ]:
deployment_info = ph.deployments.get(deployment['id'])
deployment_info

### VI. Create Streamlit App

#### 1. Get the streamlit app template

In [28]:
streamlit_template = ph.apptemplates.get('streamlit')
del streamlit_template['icon']
streamlit_template

{'id': 'streamlit',
 'name': 'Streamlit',
 'version': 'v1.12.2',
 'description': 'Streamlit turns data scripts into shareable web apps in minutes. All in Python. All for free. No frontâ€‘end experience required.',
 'docLink': 'https://docs.primehub.io/docs/primehub-app-builtin-streamlit',
 'template': {'spec': {'httpPort': 8501,
   'podTemplate': {'spec': {'containers': [{'args': ['cd $(dirname $FILE_PATH); [ -f "packages.txt" ] && apt-get update && xargs -P1 apt-get install -yq < packages.txt; [ -f "requirements.txt" ] && pip install --upgrade pip && xargs -n 1 pip install < requirements.txt; streamlit run $(FILE_PATH) --server.address 0.0.0.0 --server.baseUrlPath $(PRIMEHUB_APP_BASE_URL) --server.enableWebsocketCompression "false" --server.enableCORS "false"'],
       'command': ['bash', '-c'],
       'image': 'infuseai/primehub-apps:streamlit-v1.12.2',
       'name': 'streamlit',
       'ports': [{'containerPort': 8501,
         'name': 'http',
         'protocol': 'TCP'}]}]}},
   '

#### 2. Start the streamlit app

In [32]:
import uuid
random_id = uuid.uuid4().hex[:5]

config = {
  "templateId": "streamlit",
  "id": f"streamlit-screw-detection-{random_id}",
  "displayName": f"streamlit-screw-detection-{random_id}",
  "env": [
    {
      "name": "FILE_PATH",
      "value": "/project/{}/{}/streamlit/tutorial_screw_app.py".format(current_group.lower(), e2e_folder_name)
    },
    {
      "name": "ENDPOINT",
      "value": deployment_info['endpoint']
    }
  ],
  "instanceType": "cpu-1",
  "scope": "primehub"
}
result = ph.apps.create(config)
app = result['id']
print(result)

{'id': 'streamlit-screw-detection-d8d15'}


#### 3. Start the streamlit app

In [34]:
streamlit_app_service = ph.apps.get(app)
del streamlit_app_service['appIcon']
streamlit_app_service

{'id': 'streamlit-screw-detection-d8d15',
 'displayName': 'streamlit-screw-detection-d8d15',
 'appVersion': 'v1.12.2',
 'appName': 'streamlit',
 'appDefaultEnv': [{'name': 'FILE_PATH',
   'defaultValue': '/project/path/to/your/streamlit.py',
   'optional': False,
   'description': 'Path to the python scripts'}],
 'appTemplate': {'name': 'Streamlit',
  'docLink': 'https://docs.primehub.io/docs/primehub-app-builtin-streamlit',
  'description': 'Streamlit turns data scripts into shareable web apps in minutes. All in Python. All for free. No frontâ€‘end experience required.'},
 'groupName': 'End-to-end-tutorial',
 'instanceType': 'cpu-1',
 'instanceTypeSpec': {'name': 'cpu-1',
  'displayName': 'CPU 1',
  'cpuLimit': 1,
  'memoryLimit': 2,
  'gpuLimit': 0},
 'scope': 'primehub',
 'appUrl': 'https://c.demo.primehub.io/console/apps/streamlit-screw-detection-d8d15',
 'internalAppUrl': 'http://app-streamlit-screw-detection-d8d15:8501/console/apps/streamlit-screw-detection-d8d15',
 'svcEndpoints

### VII. Stop all service

#### 1. Stop Deployment service

In [ ]:
ph.deployments.stop(deployment_info['id'])

#### 2. Stop Streamlit Service

In [36]:
stop_streamlit_app = ph.apps.stop(app)

In [41]:
streamlit_app_service = ph.apps.get(app)
del streamlit_app_service['appIcon']
streamlit_app_service['status']

'Stopped'